# HW 5
## Jacob Holmes
### Processing Work for LP Models

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pyomo.environ as pyo

In [2]:
opt = pyo.SolverFactory('cplex')

## Problem 1

### ILP

In [3]:
model_ILP = pyo.ConcreteModel()

model_ILP.i = pyo.Set(initialize = [1,2]) #variables
model_ILP.j = pyo.Set(initialize = [1,2]) #constraints

model_ILP.c = pyo.Param(model_ILP.i, initialize={1: 5, 2: 3.5}) 
model_ILP.b = pyo.Param(model_ILP.j, initialize={1: 48, 2: 77})
model_ILP.a = pyo.Param(model_ILP.i, model_ILP.j, initialize={(1,1): 4, (1,2): 2, (2,1): 1, (2,2): 5})

model_ILP.x = pyo.Var(model_ILP.i, within=pyo.NonNegativeIntegers)

def obj_expression(m):
    return sum(m.c[i] * m.x[i] for i in m.i)

model_ILP.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

def constraint_rule(m, j):
    return sum(m.a[i,j] * m.x[i] for i in m.i) <= m.b[j]

model_ILP.Constraint1 = pyo.Constraint(model_ILP.j, rule=constraint_rule)

#model_ILP.pprint()

In [4]:
# Solution to compare with for Branch and Bound

opt.solve(model_ILP)

model_ILP.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   9.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :  11.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  83.5

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 47.0 :  48.0
          2 :  None : 73.0 :  77.0


### LP<sub>0</sub>

In [5]:
model_0 = pyo.ConcreteModel()

def initialize_model_with_data(model):
    model.i = pyo.Set(initialize = [1,2]) #variables
    model.j = pyo.Set(initialize = [1,2]) #constraints

    model.c = pyo.Param(model.i, initialize={1: 5, 2: 3.5}) 
    model.b = pyo.Param(model.j, initialize={1: 48, 2: 77})
    model.a = pyo.Param(model.i, model.j, initialize={(1,1): 4, (1,2): 2, (2,1): 1, (2,2): 5})

    model.x = pyo.Var(model.i, within=pyo.NonNegativeReals)

    def obj_expression(m):
        return sum(m.c[i] * m.x[i] for i in m.i)

    model.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

    def constraint_rule(m, j):
        return sum(m.a[i,j] * m.x[i] for i in m.i) <= m.b[j]

    model.Constraint1 = pyo.Constraint(model.j, rule=constraint_rule)

initialize_model_with_data(model_0)

#model_0.pprint()

In [6]:
results_0 = opt.solve(model_0)

model_0.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 :  9.055555555555555 :  None : False : False : NonNegativeReals
          2 :     0 : 11.777777777777777 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  86.5

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 48.0 :  48.0
          2 :  None : 77.0 :  77.0


### LP<sub>1</sub>

In [7]:
model_1 = pyo.ConcreteModel()

initialize_model_with_data(model_1)

def branch_x_2_less(m):
    return m.x[2] <= 11

model_1.Branch = pyo.Constraint(rule=branch_x_2_less)

#model_1.pprint()

In [8]:
opt.solve(model_1)

model_1.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  9.25 :  None : False : False : NonNegativeReals
          2 :     0 :  11.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 84.75

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 48.0 :  48.0
          2 :  None : 73.5 :  77.0
    Branch : Size=1
        Key  : Lower : Body : Upper
        None :  None : 11.0 :  11.0


### LP <sub>2</sub>

In [9]:
model_2 = pyo.ConcreteModel()

initialize_model_with_data(model_2)

def branch_x_2_less(m):
    return m.x[2] <= 11

model_2.Branch = pyo.Constraint(rule=branch_x_2_less)

def branch_x_1_less(m):
    return m.x[1] <= 9

model_2.Branch2 = pyo.Constraint(rule=branch_x_1_less)

#model_2.pprint()

In [10]:
opt.solve(model_2)

model_2.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   9.0 :  None : False : False : NonNegativeReals
          2 :     0 :  11.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  83.5

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 47.0 :  48.0
          2 :  None : 73.0 :  77.0
    Branch : Size=1
        Key  : Lower : Body : Upper
        None :  None : 11.0 :  11.0
    Branch2 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  9.0 :   9.0


### LP<sub>3</sub>

In [11]:
model_3 = pyo.ConcreteModel()

initialize_model_with_data(model_3)

def branch_x_2_less(m):
    return m.x[2] <= 11

model_3.Branch = pyo.Constraint(rule=branch_x_2_less)

def branch_x_1_more(m):
    return m.x[1] >= 10

model_3.Branch2 = pyo.Constraint(rule=branch_x_1_more)

#model_3.pprint()

In [12]:
opt.solve(model_3)

model_3.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  10.0 :  None : False : False : NonNegativeReals
          2 :     0 :   8.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  78.0

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 48.0 :  48.0
          2 :  None : 60.0 :  77.0
    Branch : Size=1
        Key  : Lower : Body : Upper
        None :  None :  8.0 :  11.0
    Branch2 : Size=1
        Key  : Lower : Body : Upper
        None :  10.0 : 10.0 :  None


### LP<sub>4</sub>

In [13]:
model_4 = pyo.ConcreteModel()

initialize_model_with_data(model_4)

def branch_x_2_more(m):
    return m.x[2] >= 12

model_4.Branch = pyo.Constraint(rule=branch_x_2_more)

#model_4.pprint()

In [14]:
opt.solve(model_4)

model_4.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   8.5 :  None : False : False : NonNegativeReals
          2 :     0 :  12.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  84.5

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 46.0 :  48.0
          2 :  None : 77.0 :  77.0
    Branch : Size=1
        Key  : Lower : Body : Upper
        None :  12.0 : 12.0 :  None


### LP<sub>5</sub>

In [15]:
model_5 = pyo.ConcreteModel()

initialize_model_with_data(model_5)

def branch_x_2_more(m):
    return m.x[2] >= 12

model_5.Branch = pyo.Constraint(rule=branch_x_2_more)

def branch_x_1_less(m):
    return m.x[1] <= 8

model_5.Branch2 = pyo.Constraint(rule=branch_x_1_less)

#model_5.pprint()

In [16]:
opt.solve(model_5)

model_5.display()

Model unknown

  Variables:
    x : Size=2, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   8.0 :  None : False : False : NonNegativeReals
          2 :     0 :  12.2 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 82.69999999999999

  Constraints:
    Constraint1 : Size=2
        Key : Lower : Body : Upper
          1 :  None : 44.2 :  48.0
          2 :  None : 77.0 :  77.0
    Branch : Size=1
        Key  : Lower : Body : Upper
        None :  12.0 : 12.2 :  None
    Branch2 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  8.0 :   8.0


### LP<sub>6</sub>

In [17]:
model_6 = pyo.ConcreteModel()

initialize_model_with_data(model_6)

def branch_x_2_more(m):
    return m.x[2] >= 12

model_6.Branch = pyo.Constraint(rule=branch_x_2_more)

def branch_x_1_more(m):
    return m.x[1] >= 9

model_6.Branch2 = pyo.Constraint(rule=branch_x_1_more)

#model_6.pprint()

In [18]:
results_6 = opt.solve(model_6)

#model_6.display()

print(results_6)


Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 2
  Number of nonzeros: 6
  Sense: unknown
Solver: 
- Status: ok
  User time: 0.0
  Termination condition: infeasible
  Termination message: Presolve - Infeasible.
  Error rc: 0
  Time: 0.040244102478027344



## Problem 2

In [19]:
prob_2 = pyo.ConcreteModel()

prob_2.i = pyo.Set(initialize = [1,2,3,4,5]) #variables


prob_2.c = pyo.Param(prob_2.i, initialize={1: 68, 2: 28, 3: 74, 4: 44, 5:98}) 
prob_2.a = pyo.Param(prob_2.i, initialize={1: 9, 2: 3, 3: 19, 4: 9, 5:11})

prob_2.x = pyo.Var(prob_2.i, within=pyo.Binary)

def obj_expression(m):
    return sum(m.c[i] * m.x[i] for i in m.i)

prob_2.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

def constraint_rule(m):
    return sum(m.a[i] * m.x[i] for i in m.i) <= 31

prob_2.Constraint1 = pyo.Constraint(rule=constraint_rule)

#prob_2.pprint()

In [20]:
opt.solve(prob_2)

prob_2.display()

Model unknown

  Variables:
    x : Size=5, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 210.0

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 29.0 :  31.0


## Problem 6

### Part a

#### IP

In [21]:
prob_6_IP = pyo.ConcreteModel()

prob_6_IP.i = pyo.Set(initialize = [1,2,3,4,5]) #variables


prob_6_IP.c = pyo.Param(prob_6_IP.i, initialize={1: 50, 2: 21, 3: 70, 4: 43, 5:-80}) 
prob_6_IP.a = pyo.Param(prob_6_IP.i, initialize={1: 9, 2: 3, 3: 19, 4: 9, 5:-10})

prob_6_IP.x = pyo.Var(prob_6_IP.i, within=pyo.Binary)

def obj_expression(m):
    return sum(m.c[i] * m.x[i] for i in m.i)

prob_6_IP.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

def constraint_rule(m):
    return sum(m.a[i] * m.x[i] for i in m.i) <= 32

prob_6_IP.Constraint1 = pyo.Constraint(rule=constraint_rule)

#prob_6_IP.pprint()

In [22]:
opt.solve(prob_6_IP)

prob_6_IP.display()

Model unknown

  Variables:
    x : Size=5, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 141.0

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 31.0 :  32.0


#### LP Relaxation

In [23]:
prob_6_LPR = pyo.ConcreteModel()

prob_6_LPR.i = pyo.Set(initialize = [1,2,3,4,5]) #variables


prob_6_LPR.c = pyo.Param(prob_6_LPR.i, initialize={1: 50, 2: 21, 3: 70, 4: 43, 5:-80}) 
prob_6_LPR.a = pyo.Param(prob_6_LPR.i, initialize={1: 9, 2: 3, 3: 19, 4: 9, 5:-10})

prob_6_LPR.x = pyo.Var(prob_6_LPR.i, bounds = (0,1), within=pyo.NonNegativeReals)

def obj_expression(m):
    return sum(m.c[i] * m.x[i] for i in m.i)

prob_6_LPR.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

def constraint_rule(m):
    return sum(m.a[i] * m.x[i] for i in m.i) <= 32

prob_6_LPR.Constraint1 = pyo.Constraint(rule=constraint_rule)

#prob_6_LPR.pprint()

In [24]:
opt.solve(prob_6_LPR)

prob_6_LPR.display()

Model unknown

  Variables:
    x : Size=5, Index=i
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 :                1.0 :     1 : False : False : NonNegativeReals
          2 :     0 :                1.0 :     1 : False : False : NonNegativeReals
          3 :     0 : 0.5789473684210527 :     1 : False : False : NonNegativeReals
          4 :     0 :                1.0 :     1 : False : False : NonNegativeReals
          5 :     0 :                0.0 :     1 : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 154.5263157894737

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 32.0 :  32.0


### Part b

#### IP

In [26]:
prob_6_IP_TC = pyo.ConcreteModel()

prob_6_IP_TC.i = pyo.Set(initialize = [1,2,3,4,5]) #variables


prob_6_IP_TC.c = pyo.Param(prob_6_IP_TC.i, initialize={1: 50, 2: 21, 3: 70, 4: 43, 5:-80}) 
prob_6_IP_TC.a = pyo.Param(prob_6_IP_TC.i, initialize={1: 8, 2: 3, 3: 8, 4: 8, 5:-8})

prob_6_IP_TC.x = pyo.Var(prob_6_IP_TC.i, within=pyo.Binary)

def obj_expression(m):
    return sum(m.c[i] * m.x[i] for i in m.i)

prob_6_IP_TC.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

def constraint_rule(m):
    return sum(m.a[i] * m.x[i] for i in m.i) <= 19

prob_6_IP_TC.Constraint1 = pyo.Constraint(rule=constraint_rule)

#prob_6_IP_TC.pprint()

In [27]:
opt.solve(prob_6_IP_TC)

prob_6_IP_TC.display()

Model unknown

  Variables:
    x : Size=5, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 141.0

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 19.0 :  19.0


#### LP Relaxation

In [29]:
prob_6_LPR_TC = pyo.ConcreteModel()

prob_6_LPR_TC.i = pyo.Set(initialize = [1,2,3,4,5]) #variables


prob_6_LPR_TC.c = pyo.Param(prob_6_LPR_TC.i, initialize={1: 50, 2: 21, 3: 70, 4: 43, 5:-80}) 
prob_6_LPR_TC.a = pyo.Param(prob_6_LPR_TC.i, initialize={1: 8, 2: 3, 3: 8, 4: 8, 5:-8})

prob_6_LPR_TC.x = pyo.Var(prob_6_LPR_TC.i, bounds = (0,1), within=pyo.NonNegativeReals)

def obj_expression(m):
    return sum(m.c[i] * m.x[i] for i in m.i)

prob_6_LPR_TC.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

def constraint_rule(m):
    return sum(m.a[i] * m.x[i] for i in m.i) <= 19

prob_6_LPR_TC.Constraint1 = pyo.Constraint(rule=constraint_rule)

#prob_6_LPR_TC.pprint()

In [30]:
opt.solve(prob_6_LPR_TC)

prob_6_LPR_TC.display()

Model unknown

  Variables:
    x : Size=5, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : NonNegativeReals
          2 :     0 :   1.0 :     1 : False : False : NonNegativeReals
          3 :     0 :   1.0 :     1 : False : False : NonNegativeReals
          4 :     0 :   0.0 :     1 : False : False : NonNegativeReals
          5 :     0 :   0.0 :     1 : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 141.0

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 19.0 :  19.0
